# Part 7 - Recommender Systems

## Import and Sanity Checks

In [1]:
# Checking if I'm in the right environment:
import sys
sys.executable

'C:\\Users\\mathe\\anaconda3\\envs\\deepchef\\python.exe'

In [2]:
# General libraries:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler

In [3]:
# This allows scrolling through all the columns. Useful for dataframes with too many columns.
pd.set_option('display.max_columns', None)

In [4]:
recipes = pd.read_parquet('BasicCleanData.parquet')

In [5]:
recipes = recipes.reset_index(drop=True)

In [9]:
recipes.isna().sum()

RecipeId                           0
Name                               0
AuthorId                           0
AuthorName                         0
Description                        0
Images                             0
RecipeCategory                     0
Keywords                           0
RecipeIngredientQuantities         0
RecipeIngredientParts              0
ReviewCount                   246698
Calories                           0
FatContent                         0
SaturatedFatContent                0
CholesterolContent                 0
SodiumContent                      0
CarbohydrateContent                0
FiberContent                       0
SugarContent                       0
ProteinContent                     0
RecipeServings                182733
RecipeInstructions                 0
url                                0
ingred_quants                      0
ingred_items                       0
YearPublished                      0
MonthPublished                     0
D

In [12]:
recipes.dropna(subset=['RecipeServings', 'Images'],inplace=True)

In [14]:
recipes.shape

(338974, 31)

In [15]:
recipes.isna().sum()

RecipeId                           0
Name                               0
AuthorId                           0
AuthorName                         0
Description                        0
Images                             0
RecipeCategory                     0
Keywords                           0
RecipeIngredientQuantities         0
RecipeIngredientParts              0
ReviewCount                   163124
Calories                           0
FatContent                         0
SaturatedFatContent                0
CholesterolContent                 0
SodiumContent                      0
CarbohydrateContent                0
FiberContent                       0
SugarContent                       0
ProteinContent                     0
RecipeServings                     0
RecipeInstructions                 0
url                                0
ingred_quants                      0
ingred_items                       0
YearPublished                      0
MonthPublished                     0
D

## Data Cleaning

In [8]:
nutritional_cols = ['Calories','FatContent','SaturatedFatContent','CholesterolContent','SodiumContent','CarbohydrateContent','FiberContent','SugarContent','ProteinContent','RecipeServings']
nutritional_facts = recipes_no_null[nutritional_cols]

NameError: name 'recipes_no_null' is not defined

Extract Nutritional Facts for Individuals:

In [25]:
nutritional_facts = nutritional_facts.apply(lambda column: round(column/nutritional_facts['RecipeServings'],2))
nutritional_facts

,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings
0,42.72,0.62,0.32,2.00,7.45,9.28,0.90,7.55,0.80,1.0
1,185.12,9.80,2.77,62.13,61.40,14.07,1.50,3.40,10.57,1.0
2,77.78,0.05,0.00,0.00,0.45,20.38,0.10,19.30,0.08,1.0
3,268.05,12.00,1.90,0.00,779.30,32.10,8.65,16.05,14.65,1.0
4,25.90,0.10,0.02,0.00,239.82,6.28,1.20,4.42,1.08,1.0
...,...,...,...,...,...,...,...,...,...,...
170209,82.55,0.30,0.10,0.00,3514.20,20.65,1.50,19.10,1.40,1.0
170210,285.40,0.10,0.00,0.00,6994.90,76.70,0.30,75.50,0.60,1.0
170211,44.92,2.72,0.88,71.95,127.85,1.55,0.12,0.25,3.32,1.0
170212,19.36,1.02,0.14,1.43,21.08,2.41,0.13,1.32,0.27,1.0


In [26]:
recipes_no_null.drop(nutritional_cols,axis=1,inplace=True)

In [27]:
recipes_no_null.head()

,RecipeId,AuthorId,Description,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,ReviewCount,RecipeInstructions,url,YearPublished,MonthPublished,DayPublished,HourPublished,TotalMinutes,CorrectAggregatedRating,"(Baked Goods,)","(Beans/Legumes,)","(Beverages,)","(Breakfast/Brunch,)","(Chicken,)","(Cooking Methods,)","(Dairy,)","(Desserts,)","(Family-Friendly,)","(Flavor Profiles,)","(Fruit,)","(Healthy,)","(International,)","(Lunch/Snacks,)","(Main Dish,)","(Meat,)","(Nuts/Seeds/Grains,)","(Occasions,)","(Pasta,)","(Poultry,)","(Quick and Easy,)","(Regional,)","(Sauces/Condiments,)","(Seafood,)","(Seasonal,)","(Side Dishes,)","(Soups,)","(Special Dietary Needs,)","(Uncategorized,)","(Vegetables,)","(Vegetarian/Vegan,)"
0,38.0,1533,Make and share this Low-Fat Berry Blue Frozen ...,Desserts,"[Dessert, Low Protein, Low Cholesterol, Health...","[4, 1⁄4, 1, 1]","[blueberries, granulated sugar, vanilla yogurt...",4.0,"[Toss 2 cups berries with sugar., Let stand fo...",https://www.food.com/recipe/Low-Fat-Berry-Blue...,1999,8,9,21,285,4.25,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,39.0,1567,Make and share this Biryani recipe from Food.com.,Chicken,"[Chicken Thigh & Leg, Chicken, Poultry, Meat, ...","[1, 4, 2, 2, 8, 1⁄4, 8, 1⁄2, 1, 1, 1⁄4, 1⁄4, 1...","[saffron, milk, hot green chili peppers, onion...",1.0,[Soak saffron in warm milk for 5 minutes and p...,https://www.food.com/recipe/Biryani-39,1999,8,29,13,265,3.00,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,40.0,1566,This is from one of my first Good House Keepi...,Beverages,"[Low Protein, Low Cholesterol, Healthy, Summer...","[1 1⁄2, 1, None, 1 1⁄2, None, 3⁄4]","[sugar, lemons, rind of, lemon, zest of, fresh...",10.0,"[Into a 1 quart Jar with tight fitting lid, pu...",https://www.food.com/recipe/Best-Lemonade-40,1999,9,5,19,35,4.33,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,41.0,1586,This dish is best prepared a day in advance to...,Vegetarian/Vegan,"[Beans, Vegetable, Low Cholesterol, Weeknight,...","[12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1⁄2, 1⁄...","[extra firm tofu, eggplant, zucchini, mushroom...",2.0,"[Drain the tofu, carefully squeezing out exces...",https://www.food.com/recipe/Carina's-Tofu-Vege...,1999,9,3,14,260,4.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,42.0,1538,Make and share this Cabbage Soup recipe from F...,Vegetables,"[Low Protein, Vegan, Low Cholesterol, Healthy,...","[46, 4, 1, 2, 1]","[plain tomato juice, cabbage, onion, carrots, ...",11.0,"[Mix everything together and bring to a boil.,...",https://www.food.com/recipe/Cabbage-Soup-42,1999,9,19,6,50,2.67,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [28]:
recipes_no_null = pd.concat([recipes_no_null, nutritional_facts],axis=1)
recipes_no_null.head()

,RecipeId,AuthorId,Description,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,ReviewCount,RecipeInstructions,url,YearPublished,MonthPublished,DayPublished,HourPublished,TotalMinutes,CorrectAggregatedRating,"(Baked Goods,)","(Beans/Legumes,)","(Beverages,)","(Breakfast/Brunch,)","(Chicken,)","(Cooking Methods,)","(Dairy,)","(Desserts,)","(Family-Friendly,)","(Flavor Profiles,)","(Fruit,)","(Healthy,)","(International,)","(Lunch/Snacks,)","(Main Dish,)","(Meat,)","(Nuts/Seeds/Grains,)","(Occasions,)","(Pasta,)","(Poultry,)","(Quick and Easy,)","(Regional,)","(Sauces/Condiments,)","(Seafood,)","(Seasonal,)","(Side Dishes,)","(Soups,)","(Special Dietary Needs,)","(Uncategorized,)","(Vegetables,)","(Vegetarian/Vegan,)",Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings
0,38.0,1533,Make and share this Low-Fat Berry Blue Frozen ...,Desserts,"[Dessert, Low Protein, Low Cholesterol, Health...","[4, 1⁄4, 1, 1]","[blueberries, granulated sugar, vanilla yogurt...",4.0,"[Toss 2 cups berries with sugar., Let stand fo...",https://www.food.com/recipe/Low-Fat-Berry-Blue...,1999,8,9,21,285,4.25,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,42.72,0.62,0.32,2.00,7.45,9.28,0.90,7.55,0.80,1.0
1,39.0,1567,Make and share this Biryani recipe from Food.com.,Chicken,"[Chicken Thigh & Leg, Chicken, Poultry, Meat, ...","[1, 4, 2, 2, 8, 1⁄4, 8, 1⁄2, 1, 1, 1⁄4, 1⁄4, 1...","[saffron, milk, hot green chili peppers, onion...",1.0,[Soak saffron in warm milk for 5 minutes and p...,https://www.food.com/recipe/Biryani-39,1999,8,29,13,265,3.00,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,185.12,9.80,2.77,62.13,61.40,14.07,1.50,3.40,10.57,1.0
2,40.0,1566,This is from one of my first Good House Keepi...,Beverages,"[Low Protein, Low Cholesterol, Healthy, Summer...","[1 1⁄2, 1, None, 1 1⁄2, None, 3⁄4]","[sugar, lemons, rind of, lemon, zest of, fresh...",10.0,"[Into a 1 quart Jar with tight fitting lid, pu...",https://www.food.com/recipe/Best-Lemonade-40,1999,9,5,19,35,4.33,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,77.78,0.05,0.00,0.00,0.45,20.38,0.10,19.30,0.08,1.0
3,41.0,1586,This dish is best prepared a day in advance to...,Vegetarian/Vegan,"[Beans, Vegetable, Low Cholesterol, Weeknight,...","[12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1⁄2, 1⁄...","[extra firm tofu, eggplant, zucchini, mushroom...",2.0,"[Drain the tofu, carefully squeezing out exces...",https://www.food.com/recipe/Carina's-Tofu-Vege...,1999,9,3,14,260,4.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,268.05,12.00,1.90,0.00,779.30,32.10,8.65,16.05,14.65,1.0
4,42.0,1538,Make and share this Cabbage Soup recipe from F...,Vegetables,"[Low Protein, Vegan, Low Cholesterol, Healthy,...","[46, 4, 1, 2, 1]","[plain tomato juice, cabbage, onion, carrots, ...",11.0,"[Mix everything together and bring to a boil.,...",https://www.food.com/recipe/Cabbage-Soup-42,1999,9,19,6,50,2.67,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,25.90,0.10,0.02,0.00,239.82,6.28,1.20,4.42,1.08,1.0


In [30]:
recipes_no_null = recipes_no_null.reset_index(drop=True)

## Recommender System

We can implement the recommender system using a pipeline. Here's an example of how you can use scikit-learn's pipeline to build the recommender system:

In [7]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

df = recipes_no_null.copy()

# Select the columns used for similarity calculation
selected_columns = ['Calories', 'FatContent', 'SaturatedFatContent', 'CholesterolContent',
                    'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent',
                    'ProteinContent', 'CorrectAggregatedRating']

# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Step 1: Scale the selected features
    ('knn', NearestNeighbors(metric='euclidean'))  # Step 2: Apply the KNN model with cosine similarity metric
])

# Fit the pipeline
scaled_features = pipeline['scaler'].fit_transform(df[selected_columns].values)
pipeline['knn'].fit(scaled_features)

# Function to get similar recipes
def get_similar_recipes(recipe_id, top_n=20):
    recipe_index = df[df['RecipeId'] == recipe_id].index[0]
    recipe_features = df.loc[recipe_index, selected_columns].values.reshape(1, -1)
    recipe_features_scaled = pipeline['scaler'].transform(recipe_features)
    _, top_indices = pipeline['knn'].kneighbors(recipe_features_scaled, n_neighbors=top_n+1)
    top_indices = top_indices.flatten()[1:]
    top_recipe_ids = df.loc[top_indices, 'RecipeId'].values
    top_recipes = df[df['RecipeId'].isin(top_recipe_ids)]
    return top_recipes

# Example usage:
recipe_id = 4662  # Replace with the actual RecipeId you want recommendations for
top_similar_recipes = get_similar_recipes(recipe_id, top_n=20)
pd.concat([df[df['RecipeId'] == recipe_id],get_similar_recipes(recipe_id, top_n=20)])

NameError: name 'recipes_no_null' is not defined